# Editing Data


In [1]:
import csv
import re
from datetime import datetime
from unittest import main
import pandas as pd 

- Sort news by date



In [7]:
# read files
with open('../../Data/News/news-2019-2020.csv', mode="r", encoding="ISO-8859-1") as rf:
    reader = csv.reader(rf)
    #skip header 
    next(reader)
    with open("../../Data/news_sort.csv", mode="w", encoding="ISO-8859-1") as wf:
        writer = csv.writer(wf)
        writer.writerow(['Published_date', 'Headline', 'link'])
        # change the format 
        for line in reader:
            dateTime = line[0].split() # Thu, 02 Jan 2020 08:00:00 GMT
            datetimeformat = dateTime[3] + "-" + dateTime[2] + "-" + dateTime[1] + " " + dateTime[4] #2020-Jan-02 08:00:00
            datetime64format = pd.to_datetime(datetimeformat)
            writer.writerow([datetime64format, line[1], line[2]])

In [23]:
df = pd.read_csv('../../Data/News/news2019-2021.csv')
df = df.sort_values(by="Published_date")
df.to_csv('../../Data/News/news2019-2021-2020.csv')

- Remove news name from headline

In [2]:
def remove_name(str):
    list = str.split('-')
    str = '-'.join(list[:-1])      
    return str

def remove_specialcha(str):
    list = str.split('–')
    if len(list) >= 2:
        str = '-'.join(list[:-1])
    else:
        str = list[0]     
    return str

df = pd.read_csv('../../Data/News/news2019-2021.csv')
df['Headline'] = df['Headline'].apply(remove_name).apply(remove_specialcha)
df = df.drop('link', 1)
df.to_csv('../../Data/News/news2019-2021.csv')


c:\Users\Yuji\Desktop\yxf965\venv\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  app.launch_new_instance()


- Count news per day


In [29]:
def reformatdate(df):
    newformat = []
    for index in df:
        newformat.append(pd.to_datetime(index).strftime('%y/%m/%d')) #%d-%m-%y
    return newformat

df = pd.read_csv('../../Data/Sentiment/combine_2019-2021.csv')
df['Published_date'] = reformatdate(df['Published_date'])
lst = df['Published_date'].value_counts(sort=False)
a = pd.DataFrame(lst)
a.to_csv('../../Data/newsCount_2019-2021.csv')


TypeError: sort_values() got an unexpected keyword argument 'by'

In [33]:
df = pd.read_csv('../../Data/newsCount_2019-2021.csv')
df.sort_values(by='Date',  inplace=True)
df.to_csv('../../Data/newsCount_2019-2021.csv')

- Sort crpto data by Date

In [ ]:
df = pd.read_csv('Data/Bitcoin Historical Data.csv')
for row in df.iterrows():
    dateTime = row[1][0].split() # Aug 24, 2021
    dateTime[1].replace(',', '')
    datetimeformat = dateTime[2] + "-" + dateTime[0] + "-" + dateTime[1] #2021-Aug-24
    row[1][0] = pd.to_datetime(datetimeformat, unit='D')
    
df.sort_values(by='Date',  inplace=True)
df.to_csv('Data/crypto.csv')
    

- delate time from date


In [5]:
def removetime(str):
    return  ''.join(str.split()[:-1])  

df = pd.read_csv('final_table.csv', encoding = "ISO-8859-1")
df['Published_date'] = df['Published_date'].apply(removetime)
df.to_csv('remove_time.csv', encoding = "ISO-8859-1")

- Count parametar

In [3]:
firstday = pd.to_datetime('2019-01-01')
lst = []
# read files
with open('remove_time.csv', mode="r", encoding="ISO-8859-1") as rf:
    reader = csv.reader(rf)
    next(reader)    #skip title
    # change the format 
    positive = 0
    negative = 0
    for line in reader:
        date_row = line[1]
        if pd.to_datetime(date_row) == firstday:
            if line[7] == 'NEGATIVE':
                negative = negative + 1 
            if line[7] == 'POSITIVE':
                positive = positive + 1
        else:
            if(positive - negative) >= 0:
                lst.append("POSITIVE")
            else:
                lst.append("NEGATIVE")
            positive = 0
            negative = 0
            firstday = firstday + pd.Timedelta(days=1)
    if(positive - negative) >= 0:
        lst.append(1)
    else:
        lst.append(0)   
df = pd.read_csv('../../Data/Bitcoin_2019_2020.csv', encoding = "ISO-8859-1")
df["SENTIMENTAL"] = lst
df.to_csv('../../Data/Bitcoin_2019_2020.csv', encoding = "ISO-8859-1")
        

Compare vader and flair result

In [40]:
df= pd.read_csv('../../Data/Sentiment/combine_2019-2021.csv')
combine_sentiment = []

for index, row in df.iterrows():
    if row['Sentiment_Flair'] == "NEGATIVE" and row['Sentiment_Vader'] == "POSITIVE":
        combine_sentiment.append("NEUTRAL")
    elif row['Sentiment_Flair'] == "NEGATIVE":
        combine_sentiment.append("NEGATIVE")
    elif row['Sentiment_Vader'] == "POSITIVE":
        combine_sentiment.append("POSITIVE")
    else:
         combine_sentiment.append("NEUTRAL")

df['combine_sentiment_N'] = combine_sentiment
df.to_csv('../../Data/Sentiment/combine_2019-2021.csv')

In [17]:
df1= pd.read_csv('../../Data/Training/news_val_2021Jan_noN.csv')
i = 0
for index, row in df1.iterrows():
    if row['Sentiment'] == row['combine_sentiment']:
        i  = i + 1

accuracy = i / 209
        
print(accuracy)

0.6794258373205742


In [48]:
def getpolarity(df):
    polarity = []
    for index in df:
        if index =='POSITIVE':
            polarity.append(1)
        elif index == 'NEGATIVE':
            polarity.append(-1)
        else:
            polarity.append(0)
    return polarity

def reformatdate(df):
    newformat = []
    for index in df:
        newformat.append(pd.to_datetime(index).strftime('%y/%m/%d')) #%d-%m-%y
    return newformat

#get every day avarage sentiment
df1 = pd.read_csv('../../Data/Sentiment/combine_2019-2021.csv')
df2 = pd.read_csv('../../Data/newsCount_2019-2021.csv')
df1['final'] = getpolarity(df1['combine_sentiment_N'])
df1['Published_date'] = reformatdate(df['Published_date'])
lst= df1.groupby(['Published_date']).sum()['final']
lst.to_csv('../../Data/Sentiment/senti_test.csv')

In [3]:
df= pd.read_csv('../../Data/TestData_2019_2021.csv')
nextday = []

for index, row in df.iterrows():
    if row['change in next day (%)'] < 0:
        nextday.append("NEGATIVE")
    else :
        nextday.append("POSITIVE")

df['nextDay_prediction'] = nextday
df.to_csv('../../Data/TestData_2019_2021.csv')